<a href="https://colab.research.google.com/github/japb11/tinyv4/blob/main/tiny.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
from torchsummary import summary
from torchcontrib.optim import SWA

_INPUT_SIZE = 416

_ANCHORS = [(23, 27), (37, 58), (81, 82),
            (81, 82), (135, 169), (344, 319)]

ModuleNotFoundError: ignored

In [ ]:
#take a look at the kind of GPU we have
!nvidia-smi

Mon Feb 22 23:39:00 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P0    31W /  70W |   3058MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
class BasicConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1):
        super(BasicConv, self).__init__()

        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, kernel_size//2, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.activation = nn.LeakyReLU(0.1)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.activation(x)
        return x

In [ ]:
class CSPBlock(nn.Module):
    def __init__(self, in_channels):
      super(CSPBlock, self).__init__()
      self.input_size = in_channels
      
      #DenseBlock
      #->1x1 conv
      #->3x3 conv
      #Transition Layer
      #->1 x 1 conv layer
      #-> 2 x 2 pooling layer, S=2

      self.conv_1 = BasicConv(in_channels, in_channels, 3, 1)
      self.conv_2 = BasicConv(in_channels // 2, in_channels // 2, 3, 1)
      self.conv_3 = BasicConv(in_channels // 2, in_channels // 2, 3, 1)
      self.conv_4 = BasicConv(in_channels, in_channels, 1, 1)
      self.maxpool = nn.MaxPool2d([2,2],[2,2])
    
    def forward(self, x):

      size = self.input_size

      x = self.conv_1(x)
      
      x_1 = x
      
      x_2 = torch.split(x, size // 2, 1)[1]
      
      
      
      x_2 = self.conv_2(x_2)
      x_3 = x_2
      #x_4 = torch.split(x_2, size // 2, 1)[1]

      x_2 = self.conv_3(x_2)
      
      x_2 = torch.cat((x_2, x_3), 1)
      
      x_2 = self.conv_4(x_2)
      
      x = torch.cat((x_1, x_2), 1)
      
      x = self.maxpool(x)
      
      return x


In [ ]:
class YOLO_head(nn.Module):
  def __init__(self, in_channels, anchors, n_classes):
    super(YOLO_head, self).__init__()
    self.anchors = nn.Parameter(
            torch.FloatTensor(anchors).view(-1, 1, 1, 2), requires_grad=False
        )
    self.num_anchors = len(anchors)
    self.num_classes = n_classes
    self.conv = nn.Conv2d(in_channels, self.num_anchors * (5 + self.num_classes), 1, 1, bias=False)

  def forward(self, input, img_size):
    
    pred = self.conv(input)
    
    B = pred.size(0)
    H = pred.size(2)
    W = pred.size(3)
    N = self.num_classes

    stride = img_size[0] // H

    grid_x = torch.arange(W, device=device).repeat(H, 1).to(torch.float)
    grid_y = torch.arange(H, device=device).repeat(W, 1).t().to(torch.float)
    grid = torch.stack([grid_x, grid_y], dim=2)
    grid = grid.data
    pred = pred.view(B, -1, N + 5, H, W)
    pred = pred.permute(0, 1, 3, 4, 2)
    xy, wh, box_conf, cls_conf = torch.split(pred, [2, 2, 1, N], dim=-1)

    xy = stride * (torch.sigmoid(xy) + grid)
    wh = torch.exp(wh) * self.anchors
    boxes = torch.cat([xy, wh], dim=-1)
    boxes = boxes.view(B, -1, 4)
    box_conf = torch.sigmoid(box_conf).view(B, -1, 1)
    cls_conf = torch.sigmoid(cls_conf).reshape(B, -1, N)
 
    return torch.cat([boxes, box_conf, cls_conf], dim=-1)

In [ ]:
#GFLv2 HEAD

In [ ]:
# _________________
# | Input 416x416 |
# |_______________|
#        |
#   _____|____
#   | conv_1 |
#   |________|
#   _____|____
#   | conv_2 |
#   |________|
#   _____|____
#   | cspb_1 |
#   |________|
#   _____|____        ________         ________        ________ 
#   | cspb_2 |-------|   cat  |-------| conv_6 |------| YOLO   |
#   |________|       |________|       |________|      |________|
#   _____|____        ____|____
#   | cspb_3 |       | Upsample|
#   |________|       |_________|
#        |           |  conv_5 |
#        |           |_________|
#   _____|____        ____|____         ________        ________ 
#   | conv_3 |-------| conv_4  |-------| conv_7 |------| YOLO   |
#   |________|       |_________|       |________|      |________|

In [ ]:
#Creating Class where it will be defined the tiny-YoloV4 architecture
class tiny_YOLOV4(nn.Module):
  def __init__(self):
    super(tiny_YOLOV4, self).__init__()

    self.conv_1 = BasicConv(3, 32, 3, 2)
    self.conv_2 = BasicConv(32, 64, 3, 2)

    self.cspb_1 = CSPBlock(64)
    self.cspb_2 = CSPBlock(128)

    self.cspb_3 = CSPBlock(256)

    self.conv_3 = BasicConv(512, 512, 3, 1)

    self.conv_4 = BasicConv(512, 512, 3, 1)

    
    self.conv_5 = nn.Conv2d(512, 256, 1, 1)
    self.upsample = nn.Upsample(scale_factor=2, mode='nearest')

    self.conv_6 = BasicConv(512, 256, 3, 1)
    self.conv_7 = BasicConv(512, 512, 3, 1)
    self.yolo_1 = YOLO_head(256, _ANCHORS[:3] ,80)
    self.yolo_2 = YOLO_head(512, _ANCHORS[-3:] ,80)
    #self.conv_6 = BasicConv(512, 128, 1, 1)

  def forward(self, x):
    input = x
    x = self.conv_1(input)
    x = self.conv_2(x)
    x = self.cspb_1(x)
    x = self.cspb_2(x)
    x_1 = x
    x = self.cspb_3(x)
    x = self.conv_3(x)
    x = self.conv_4(x)
    x_2 = x
    x = self.conv_5(x)
    x = self.upsample(x)
    
    x_1 = torch.cat((x_1, x), 1)
    x_1 = self.conv_6(x_1)
    x_2 = self.conv_7(x_2)
    yolo_res_1 = self.yolo_1(x_1, input.shape[-2:])
    yolo_res_2 = self.yolo_2(x_2, input.shape[-2:])
    return yolo_res_1, yolo_res_2

In [ ]:
#!gdown https://github.com/bubbliiiing/yolov4-tiny-pytorch/releases/download/v1.0/yolov4_tiny_weights_coco.pth

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = tiny_YOLOV4().to(device)
summary(model, input_size=(3, 416, 416))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 208, 208]             864
       BatchNorm2d-2         [-1, 32, 208, 208]              64
         LeakyReLU-3         [-1, 32, 208, 208]               0
         BasicConv-4         [-1, 32, 208, 208]               0
            Conv2d-5         [-1, 64, 104, 104]          18,432
       BatchNorm2d-6         [-1, 64, 104, 104]             128
         LeakyReLU-7         [-1, 64, 104, 104]               0
         BasicConv-8         [-1, 64, 104, 104]               0
            Conv2d-9         [-1, 64, 104, 104]          36,864
      BatchNorm2d-10         [-1, 64, 104, 104]             128
        LeakyReLU-11         [-1, 64, 104, 104]               0
        BasicConv-12         [-1, 64, 104, 104]               0
           Conv2d-13         [-1, 32, 104, 104]           9,216
      BatchNorm2d-14         [-1, 32, 1

In [ ]:
#train the network
class Trainer():
  def __init__(self, ep_train = 100, ep_val, lr = 1e-1, wd = 1e-5):
    self.num_epochs_train = ep_train
    self.num_epochs_val = ep_val 
    self.lr = lr
    self.wd = wd

  def train(model):

    """ define loss criterion """
    criterion = nn.CrossEntropyLoss().cuda()

    """ define optimizer """
    optimizer = torch.optim.AdamW(model.parameters(), lr=self.lr, weight_decay=1e-5)

    """ define learning rate scheduler """
    scheduler = torch.optim.lr_scheduler.CyclicLRWithRestarts(optimizer, self.Batch_size, epoch_size, restart_period=5, t_mult=1.2, policy="cosine")

    """ define loss scaler for automatic mixed precision """
    scaler = torch.cuda.amp.GradScaler()

    """ define model and learning rate scheduler for stochastic weight averaging """
    swa_model = torch.optim.swa_utils.AveragedModel(model)
    swa_scheduler = torch.optim.swa_utils.SWALR(optimizer, swa_lr=5e-5)


    for epoch in range(100):
          for input, target in loader:
              optimizer.zero_grad()
              loss_fn(model(input), target).backward()
              optimizer.step()
          if epoch > swa_start:
              swa_model.update_parameters(model)
              swa_scheduler.step()
          else:
              scheduler.step()

    for input, target in loader:
      optimizer.zero_grad()
      loss_fn(model(input), target).backward()
      optimizer.step()
      if epoch > swa_start:
          swa_model.update_parameters(model)
          swa_scheduler.step()
      else:
          scheduler.step()


    for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

  print('Finished Training')
  

SyntaxError: ignored